# Capstone Project — Python Fundamentals (Annotated Solution)

This notebook is an **annotated** and **cell-by-cell** walkthrough of the capstone pipeline.  
It **generates synthetic data** (≥100 employees), **saves CSVs**, then **executes Tasks 2–10**, and finally **writes outputs**.

**What you'll find here:**  
1. Setup & paths  
2. Synthetic data generation (Employees, Seniority, Projects)  
3. Persist the CSVs (Task 1)  
4. Data processing pipeline (Tasks 2–10): running-average imputation, joins, bonus, demotion/promotion rules, totals  
5. Save final outputs and preview


## 1) Setup & Paths
We set a base folder (`/mnt/data`) for all reads/writes and define file paths for the input/output CSVs and notebook artifacts.
This ensures a clean, reproducible location for everything the notebook produces.


In [1]:
# --- Setup & Paths ---
from pathlib import Path

OUT_DIR = Path("data")         # Where we'll save inputs/outputs
OUT_DIR.mkdir(parents=True, exist_ok=True)

EMP_PATH = OUT_DIR / "employee.csv"
SEN_PATH = OUT_DIR / "seniority.csv"
PROJ_PATH = OUT_DIR / "project.csv"
FINAL_PATH = OUT_DIR / "Final.csv"
TPC_PATH = OUT_DIR / "TotalProjCost.csv"

# No output here — just path setup.


## 2) Generate Synthetic Data — Employees
We create **≥100 employees** with random:
- `ID` (A001, A002, ...),  
- `Name` (gender-specific first name + last name),  
- `Gender`, `City`, and `Age`.

> This mirrors the project’s requirement to start from three dataframes and then save them to CSVs.


In [2]:
# --- Synthetic Employees ---
import random
import numpy as np
import pandas as pd

random.seed(7)
np.random.seed(7)

N_EMP = 120  # ensure >= 100
EMP_IDS = [f"A{idx:03d}" for idx in range(1, N_EMP + 1)]

first_names_m = ["John","Tom","Alex","Michael","David","Chris","Robert","Daniel","Mark","Paul","Ryan","Kevin","Brian","Jason"]
first_names_f = ["Alice","Nina","Amy","Sophia","Emma","Olivia","Mia","Ava","Isabella","Emily","Charlotte","Grace","Lily","Zoe"]
last_names = ["Alter","Luxumberg","Sabestine","Adgra","Johny","Smith","Brown","Taylor","Anderson","Thomas","Jackson","White","Harris","Martin","Clark","Lewis"]
cities = ["Paris","London","Berlin","New York","Madrid","Rome","Vienna","Prague","Warsaw","Zurich","Lisbon","Dublin","Oslo","Stockholm","Helsinki","Copenhagen"]
genders = ["M","F"]

# Ages between 22 and 55 for variety
ages = np.random.randint(22, 56, size=N_EMP)

def rand_name(g):
    """Pick a gender-appropriate first name and a random last name."""
    if g == "M":
        f = random.choice(first_names_m)
    else:
        f = random.choice(first_names_f)
    l = random.choice(last_names)
    return f"{f} {l}", f, l

emp_rows = []
for i, emp_id in enumerate(EMP_IDS):
    g = random.choice(genders)
    full, f, l = rand_name(g)
    city = random.choice(cities)
    age = int(ages[i])
    emp_rows.append([emp_id, full, g, city, age])

df_employee = pd.DataFrame(emp_rows, columns=["ID","Name","Gender","City","Age"])
df_employee.head(10)  # preview


,ID,Name,Gender,City,Age
0,A001,Amy Harris,F,London,26
1,A002,Jason Adgra,M,Dublin,47
2,A003,Mark Brown,M,London,25
3,A004,Robert Martin,M,Berlin,41
4,A005,Tom Martin,M,London,45
5,A006,Michael Luxumberg,M,Oslo,50
6,A007,Michael Luxumberg,M,Madrid,36
7,A008,Mia Johny,F,New York,45
8,A009,Isabella Smith,F,New York,30
9,A010,Chris Adgra,M,Berlin,47


## 3) Generate Synthetic Data — Seniority
We assign each employee an initial **Designation Level** ∈ {1,2,3,4}.  
This mirrors the separate seniority table described in the spec.


In [3]:
# --- Synthetic Seniority ---
# Bias the levels slightly toward 2 and 3
sen_levels = np.random.choice([1,2,3,4], size=N_EMP, p=[0.2,0.35,0.30,0.15])
df_seniority = pd.DataFrame({"ID": EMP_IDS, "Designation Level": sen_levels})
df_seniority.head(10)  # preview


,ID,Designation Level
0,A001,4
1,A002,2
2,A003,3
3,A004,2
4,A005,3
5,A006,1
6,A007,3
7,A008,3
8,A009,2
9,A010,3


## 4) Generate Synthetic Data — Projects
We generate **2–5 projects per employee**.  
- `Cost` is log-normally distributed and **~12%** of rows are set to **NaN** to simulate missing data.  
- `Status` ∈ {Finished, Ongoing, Failed} with a bias toward **Finished**.

This aligns with the project table that includes missing costs and mixed statuses.


In [4]:
# --- Synthetic Projects ---
status_choices = ["Finished","Ongoing","Failed"]
project_rows = []
project_counter = 1

for emp_id in EMP_IDS:
    k = random.randint(2,5)  # projects per employee
    for _ in range(k):
        proj_name = f"Project {project_counter}"
        project_counter += 1
        # Base cost from lognormal: produces positive skew similar to real costs
        base = float(np.random.lognormal(mean=13.0, sigma=0.6))
        cost = max(100_000.0, min(base, 8_000_000.0))  # clamp to reasonable range
        
        # Introduce ~12% missingness
        if random.random() < 0.12:
            cost_val = np.nan
        else:
            cost_val = round(cost, 2)
        
        # More Finished than Ongoing, more Ongoing than Failed
        status = random.choices(status_choices, weights=[0.55, 0.30, 0.15])[0]
        project_rows.append([emp_id, proj_name, cost_val, status])

df_project = pd.DataFrame(project_rows, columns=["ID","Project","Cost","Status"])
df_project.head(10)  # preview


,ID,Project,Cost,Status
0,A001,Project 1,437715.81,Finished
1,A001,Project 2,1509324.43,Failed
2,A001,Project 3,544828.78,Finished
3,A001,Project 4,1068008.47,Finished
4,A002,Project 5,395519.27,Finished
5,A002,Project 6,NaN,Finished
6,A002,Project 7,409724.78,Finished
7,A002,Project 8,429023.35,Finished
8,A003,Project 9,520937.74,Ongoing
9,A003,Project 10,452793.26,Finished


## 5) Task 1 — Save the Three Base Tables as CSVs
Per the instructions, from **Task 2 onwards** the pipeline should **read from CSVs** (not in-memory objects).


In [5]:
# --- Task 1: Save CSVs ---
df_employee.to_csv(EMP_PATH, index=False)
df_seniority.to_csv(SEN_PATH, index=False)
df_project.to_csv(PROJ_PATH, index=False)

print("Saved:", EMP_PATH.name, SEN_PATH.name, PROJ_PATH.name)


Saved: employee.csv seniority.csv project.csv


## 6) Reload the CSVs and Normalize Types
We reload the saved CSVs to simulate a fresh start, and ensure `Cost` is numeric (coercing bad values to NaN).  
This sets the stage for the imputation step.


In [6]:
# --- Reload from CSVs for Tasks 2–10 ---
emp = pd.read_csv(EMP_PATH)
sen = pd.read_csv(SEN_PATH)
proj = pd.read_csv(PROJ_PATH)

# Ensure numeric for 'Cost' (invalid strings -> NaN)
proj["Cost"] = pd.to_numeric(proj["Cost"], errors="coerce")

emp.head(3), sen.head(3), proj.head(3)


(     ID         Name Gender    City  Age
 0  A001   Amy Harris      F  London   26
 1  A002  Jason Adgra      M  Dublin   47
 2  A003   Mark Brown      M  London   25,
      ID  Designation Level
 0  A001                  4
 1  A002                  2
 2  A003                  3,
      ID    Project        Cost    Status
 0  A001  Project 1   437715.81  Finished
 1  A001  Project 2  1509324.43    Failed
 2  A001  Project 3   544828.78  Finished)

## 7) Task 2 — Running-Average Imputation (with a `for` loop)
We fill each missing `Cost` with the **average of all previously seen** costs (including earlier imputations) **in row order**.  
This is done explicitly via a `for` loop to follow the requirement.


In [7]:
# --- Task 2: Running-average imputation ---
running_sum = 0.0
running_count = 0
costs = proj["Cost"].tolist()

for i in range(len(costs)):
    if pd.isna(costs[i]):
        avg = running_sum / running_count if running_count > 0 else 0.0
        costs[i] = avg
    running_sum += float(costs[i])
    running_count += 1

proj["Cost"] = costs
proj["Cost"].isna().sum(), proj.head(10)


(np.int64(0),
      ID     Project         Cost    Status
 0  A001   Project 1   437715.810  Finished
 1  A001   Project 2  1509324.430    Failed
 2  A001   Project 3   544828.780  Finished
 3  A001   Project 4  1068008.470  Finished
 4  A002   Project 5   395519.270  Finished
 5  A002   Project 6   791079.352  Finished
 6  A002   Project 7   409724.780  Finished
 7  A002   Project 8   429023.350  Finished
 8  A003   Project 9   520937.740   Ongoing
 9  A003  Project 10   452793.260  Finished)

## 8) Task 3 — Split `Name` into `First Name` / `Last Name`
We perform `str.split` on the first space and **drop** the original `Name` column.


In [8]:
# --- Task 3: Split Name ---
emp[["First Name","Last Name"]] = emp["Name"].str.split(" ", n=1, expand=True)
emp = emp.drop(columns=["Name"])
emp.head(5)


,ID,Gender,City,Age,First Name,Last Name
0,A001,F,London,26,Amy,Harris
1,A002,M,Dublin,47,Jason,Adgra
2,A003,M,London,25,Mark,Brown
3,A004,M,Berlin,41,Robert,Martin
4,A005,M,London,45,Tom,Martin


## 9) Task 4 — Join Employee, Seniority, and Projects
We left-join `Employee → Seniority` on `ID`, then left-join `Projects` on `ID` (one-to-many).  
The result, `final`, is our **wide** per-project view with employee attributes attached.


In [9]:
# --- Task 4: Joins ---
final = emp.merge(sen, on="ID", how="left").merge(proj, on="ID", how="left")
final.head(10)


,ID,Gender,City,Age,First Name,Last Name,Designation Level,Project,Cost,Status
0,A001,F,London,26,Amy,Harris,4,Project 1,437715.810,Finished
1,A001,F,London,26,Amy,Harris,4,Project 2,1509324.430,Failed
2,A001,F,London,26,Amy,Harris,4,Project 3,544828.780,Finished
3,A001,F,London,26,Amy,Harris,4,Project 4,1068008.470,Finished
4,A002,M,Dublin,47,Jason,Adgra,2,Project 5,395519.270,Finished
5,A002,M,Dublin,47,Jason,Adgra,2,Project 6,791079.352,Finished
6,A002,M,Dublin,47,Jason,Adgra,2,Project 7,409724.780,Finished
7,A002,M,Dublin,47,Jason,Adgra,2,Project 8,429023.350,Finished
8,A003,M,London,25,Mark,Brown,3,Project 9,520937.740,Ongoing
9,A003,M,London,25,Mark,Brown,3,Project 10,452793.260,Finished


## 10) Task 5 — Bonus (5% of Cost) for `Finished` Projects Only
We add a `Bonus` column equal to `0.05 * Cost` if `Status == 'Finished'`, else `0`.


In [10]:
# --- Task 5: Bonus ---
final["Bonus"] = np.where(final["Status"].str.lower() == "finished", 0.05 * final["Cost"], 0.0)
final[["ID","Project","Status","Cost","Bonus"]].head(10)


,ID,Project,Status,Cost,Bonus
0,A001,Project 1,Finished,437715.810,21885.7905
1,A001,Project 2,Failed,1509324.430,0.0000
2,A001,Project 3,Finished,544828.780,27241.4390
3,A001,Project 4,Finished,1068008.470,53400.4235
4,A002,Project 5,Finished,395519.270,19775.9635
5,A002,Project 6,Finished,791079.352,39553.9676
6,A002,Project 7,Finished,409724.780,20486.2390
7,A002,Project 8,Finished,429023.350,21451.1675
8,A003,Project 9,Ongoing,520937.740,0.0000
9,A003,Project 10,Finished,452793.260,22639.6630


## 11) Task 6 — Demote for Any Failed Project; Remove Level > 4
- For any employee with **at least one** `Failed` project, **demote by 1** (increase the level number).  
- **Remove** rows where `Designation Level > 4` after demotion.


In [11]:
# --- Task 6: Demote + Prune ---
failed_by_emp = final.groupby("ID")["Status"].apply(lambda s: (s.str.lower() == "failed").any())

def adjust_designation(row):
    demote = 1 if failed_by_emp.get(row["ID"], False) else 0
    return row["Designation Level"] + demote

final["Designation Level"] = final.apply(adjust_designation, axis=1)
final = final[final["Designation Level"] <= 4].reset_index(drop=True)

final[["ID","Designation Level","Status"]].head(10)


,ID,Designation Level,Status
0,A002,2,Finished
1,A002,2,Finished
2,A002,2,Finished
3,A002,2,Finished
4,A003,3,Ongoing
5,A003,3,Finished
6,A004,3,Failed
7,A004,3,Finished
8,A004,3,Ongoing
9,A005,3,Ongoing


## 12) Task 7 — Prefix `Mr.`/`Mrs.` to `First Name` and Drop `Gender`
We add a courtesy prefix (based on `Gender`) and then remove the `Gender` column.


In [12]:
# --- Task 7: Prefix + Drop Gender ---
prefix = np.where(final["Gender"].str.upper()=="M", "Mr. ", "Mrs. ")
final["First Name"] = prefix + final["First Name"]
final = final.drop(columns=["Gender"])

final[["ID","First Name","Last Name"]].head(10)


,ID,First Name,Last Name
0,A002,Mr. Jason,Adgra
1,A002,Mr. Jason,Adgra
2,A002,Mr. Jason,Adgra
3,A002,Mr. Jason,Adgra
4,A003,Mr. Mark,Brown
5,A003,Mr. Mark,Brown
6,A004,Mr. Robert,Martin
7,A004,Mr. Robert,Martin
8,A004,Mr. Robert,Martin
9,A005,Mr. Tom,Martin


## 13) Task 8 — Promote by 1 if `Age > 29` (cap at Level 1)
This improves the level (numerically **smaller**), with a floor at 1.


In [13]:
# --- Task 8: Promotion rule ---
def promote_if_age(row):
    if row["Age"] > 29:
        return max(1, row["Designation Level"] - 1)
    return row["Designation Level"]

final["Designation Level"] = final.apply(promote_if_age, axis=1)
final[["ID","Age","Designation Level"]].head(10)


,ID,Age,Designation Level
0,A002,47,1
1,A002,47,1
2,A002,47,1
3,A002,47,1
4,A003,25,3
5,A003,25,3
6,A004,41,2
7,A004,41,2
8,A004,41,2
9,A005,45,2


## 14) Task 9 — Total Project Cost per Employee
We group by employee (`ID`, `First Name`) and sum `Cost` → `Total cost`.


In [14]:
# --- Task 9: Totals ---
totals = (final.groupby(["ID","First Name"], as_index=False)["Cost"].sum()
               .rename(columns={"Cost":"Total cost"}))
TotalProjCost = totals[["ID","First Name","Total cost"]].copy()

TotalProjCost.head(10)


,ID,First Name,Total cost
0,A002,Mr. Jason,2.025347e+06
1,A003,Mr. Mark,9.737310e+05
2,A004,Mr. Robert,1.083562e+06
3,A005,Mr. Tom,2.267973e+06
4,A006,Mr. Michael,2.419207e+06
5,A007,Mr. Michael,1.365218e+06
6,A008,Mrs. Mia,2.226471e+06
7,A009,Mrs. Isabella,9.677693e+05
8,A010,Mr. Chris,5.730121e+05
9,A011,Mr. Paul,1.066922e+06


## 15) Task 10 — Save Final Outputs and Preview
- Save `final` and `TotalProjCost` to CSV files.  
- Build a **unique employee view** and filter for cities containing `'o'` (case-insensitive) to satisfy the display requirement.


In [15]:
# --- Save outputs ---
final.to_csv(FINAL_PATH, index=False)
TotalProjCost.to_csv(TPC_PATH, index=False)

# --- Build employee-level unique view for city filter ---
employee_details_cols = ["ID","First Name","Last Name","City","Age","Designation Level"]
emp_details_unique = final[employee_details_cols].drop_duplicates()

mask_o = emp_details_unique["City"].str.contains("o", case=False, na=False)
emp_details_with_o = emp_details_unique[mask_o].reset_index(drop=True)

print("Saved:", FINAL_PATH.name, TPC_PATH.name)
print("\n=== Final (first 8 rows) ===\n", final.head(8))
print("\n=== TotalProjCost (first 8 rows) ===\n", TotalProjCost.head(8))
print("\n=== Employee details (City contains 'o') ===\n", emp_details_with_o.head(12))


Saved: Final.csv TotalProjCost.csv

=== Final (first 8 rows) ===
      ID    City  Age  First Name Last Name  Designation Level     Project  \
0  A002  Dublin   47   Mr. Jason     Adgra                  1   Project 5   
1  A002  Dublin   47   Mr. Jason     Adgra                  1   Project 6   
2  A002  Dublin   47   Mr. Jason     Adgra                  1   Project 7   
3  A002  Dublin   47   Mr. Jason     Adgra                  1   Project 8   
4  A003  London   25    Mr. Mark     Brown                  3   Project 9   
5  A003  London   25    Mr. Mark     Brown                  3  Project 10   
6  A004  Berlin   41  Mr. Robert    Martin                  2  Project 11   
7  A004  Berlin   41  Mr. Robert    Martin                  2  Project 12   

          Cost    Status       Bonus  
0  395519.2700  Finished  19775.9635  
1  791079.3520  Finished  39553.9676  
2  409724.7800  Finished  20486.2390  
3  429023.3500  Finished  21451.1675  
4  520937.7400   Ongoing      0.0000  
5  452